In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

import mlflow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
 
import warnings
warnings.filterwarnings('ignore')

In [38]:
#!export MLFLOW_TRACKING_URL="http://mlflow-auth-tracon-xxiv-mbahmani-0.apps.ocp.solutioncenter-munich.de"
!export MLFLOW_TRACKING_USERNAME=user
!export MLFLOW_TRACKING_PASSWORD=user

352053.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
352058.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [11]:
# Load Model from mlflow in loaded_model
MODEL_NAME = "ElasticnetWineModel"
MODEL_VERSION= 1
MODEL_URI = f"models:/{MODEL_NAME}/{MODEL_VERSION}"
loaded_model = mlflow.pyfunc.load_model(MODEL_URI)

In [9]:
# Test the performance of the Model, which is loaded in the last step, on 6 data points
df_test = pd.read_csv('winequality-red-test.csv')
df_test['best quality'] = [1 if x > 5 else 0 for x in df_test.quality]
#print(df_test.head())
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6 non-null      float64
 1   volatile acidity      6 non-null      float64
 2   citric acid           6 non-null      float64
 3   residual sugar        6 non-null      float64
 4   chlorides             6 non-null      float64
 5   free sulfur dioxide   6 non-null      int64  
 6   total sulfur dioxide  6 non-null      int64  
 7   density               6 non-null      float64
 8   pH                    6 non-null      float64
 9   sulphates             6 non-null      float64
 10  alcohol               6 non-null      float64
 11  quality               6 non-null      int64  
 12  best quality          6 non-null      int64  
dtypes: float64(9), int64(4)
memory usage: 752.0 bytes


In [12]:
df_test_input = df_test.drop(['quality', 'best quality'], axis=1)

# Get the actual class of six input example
df_test_actual_class = df_test['best quality']

actual_class_test = pd.Series(df_test_actual_class)
#print(actual_class_test)

# Use the trained model to predict the class of six input example
prediction_class_test = pd.DataFrame(loaded_model.predict(df_test_input))
#print(prediction_class_test)
#model_testing_output = model_logreg.predict(df_test_input), columns=['Input ID','Predicted Class'])

# Put these information together
model_output = pd.concat([prediction_class_test, actual_class_test], axis=1, ignore_index=True)

#model_output.info()
model_output = model_output.set_axis(['Predicted Class', 'Actual Class'], axis='columns')
#model_output.info()

print(model_output)


   Predicted Class  Actual Class
0                0             0
1                1             0
2                0             0
3                1             1
4                1             1
5                1             1


In [15]:
!mlflow models generate-dockerfile -m "{MODEL_URI}" --env-manager local -d wine_classifier

2024/08/02 08:05:06 INFO mlflow.models.cli: Generating Dockerfile for model models:/ElasticnetWineModel/1
2024/08/02 08:05:07 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/08/02 08:05:07 INFO mlflow.models.cli: Generated Dockerfile in directory wine_classifier


In [9]:
import requests

inference_request = {
    "dataframe_split": {
        "columns": [
            "fixed acidity",
            "volatile acidity",
            "citric acid",
            "residual sugar",
            "chlorides",
            "free sulfur dioxide",
            "total sulfur dioxide",
            "density",
            "pH",
            "sulphates",
            "alcohol",
        ],
        "data": [[7.3,0.34,0.33,2.5,0.064,21,37,0.9952,3.35,0.77,12.1]]
    }
}

endpoint = "http://127.0.0.1:8080/invocations"
# endpoint = "http://test1111-mlflow.apps.cluster-vnqz5.vnqz5.sandbox1919.opentlc.com/invocations"
response = requests.post(endpoint, json=inference_request)

response.json()

InvalidSchema: No connection adapters were found for 'file:///mnt/d/Projekts/MLOps/WbeRZ_UseCase/mlruns/654892709162939613/3f029873670f4d59af0743f8857d843f/artifacts/model_logreg/invocations'